<a href="https://colab.research.google.com/github/NastyaPorunova/gpb_test/blob/main/%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B51.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Имеется таблица сделок по следующему формату. Строк в таблице за один торговый день может быть миллионы, время сделки определяется с точностью до миллисекунды, сделки попадают в таблицу строго по хронологии, цена сделки формируется за счет заявок на покупку и продажу рыночных участников. В случае несовпадения мнений продавцов и покупателей относительно цены сделки могут определенный период не проводиться, пока не будет найден компромисс, что указывает на неоднородность периодов между сделками.
№сделки;Названиеакции;Время;Цена;Количествобумаг
Задача: Написать скрипт по поиску мошеннической сделки по критерию – отклонение от рыночной цены более чем на 10 %




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import statistics as stat

Mounted at /content/gdrive


In [ ]:
data = pd.io.excel.read_excel(r'/content/газпром 2023 апрель.xlsx')
data

,номер сделки,название акции,время,цена,кол во бумаг
0,34024,тотошка,NaN,100,10
1,34024,тотошка,NaN,150,10
2,34024,тотошка,NaN,345,10
3,34024,тотошка,NaN,100,10
4,34024,тотошка,NaN,332,10
5,34024,тотошка,NaN,1354,10
6,34024,тотошка,NaN,1231,10
7,34024,тотошка,NaN,432,10
8,15312,апельсин,NaN,255,10
9,15312,апельсин,NaN,10,15


In [ ]:
data1 = data.groupby("название акции").agg({"цена":['mean','std']})
data1

цена            
                 mean         std
название акции                   
апельсин         99.0  123.960567
тотошка         505.5  501.826379

In [ ]:
fraudulent_transactions = pd.DataFrame()
stock_names = data['название акции'].unique()
stock_names

array(['тотошка', 'апельсин'], dtype=object)

In [ ]:
data2 = data[data['название акции'] == 'апельсин'].tail(11)
data2

,номер сделки,название акции,время,цена,кол во бумаг
8,15312,апельсин,NaN,255,10
9,15312,апельсин,NaN,10,15
10,15312,апельсин,NaN,15,15
11,15312,апельсин,NaN,16,15
12,15312,апельсин,NaN,333,15
13,15312,апельсин,NaN,5,15
14,15312,апельсин,NaN,234,15
15,15312,апельсин,NaN,43,15
16,15312,апельсин,NaN,45,15
17,15312,апельсин,NaN,34,15


In [ ]:
data = pd.io.excel.read_excel(r'/content/газпром 2023 апрель.xlsx')
fraudulent_transactions = pd.DataFrame()
stock_names = data['название акции'].unique()
n = ? #сколько последних сделок учитывать в подстчете рыночной цены акции; может варьироваться

for name in stock_names:
  df = data[data['название акции'] == name].tail(n).reset_index(drop=True)
  mean = df['цена'].mean()
  for i in range(n):
    if ((df['цена'][i] - mean) / mean * 100 > 10):
      fraudulent_transactions = fraudulent_transactions.append(df.loc[i])

fraudulent_transactions


<ipython-input-95-4f8fd1fcc745>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fraudulent_transactions = fraudulent_transactions.append(df.loc[i])
<ipython-input-95-4f8fd1fcc745>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fraudulent_transactions = fraudulent_transactions.append(df.loc[i])
<ipython-input-95-4f8fd1fcc745>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fraudulent_transactions = fraudulent_transactions.append(df.loc[i])
<ipython-input-95-4f8fd1fcc745>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fraudulent_transactions = fraudulent_transactions.append(df.loc[i])


,номер сделки,название акции,время,цена,кол во бумаг
4,34024,тотошка,NaN,1354,10
5,34024,тотошка,NaN,1231,10
1,15312,апельсин,NaN,333,15
3,15312,апельсин,NaN,234,15


In [ ]:
import xml.etree.ElementTree as ET 
root_node = ET.parse('/content/VO_RRMSPSV_0000_9965_20230210_fff8880e-2b57-4935-a4f3-1b31139dbc65.xml').getroot() 
print(root_node)


<Element 'Файл' at 0x7fd65447d310>


Дана выгрузка учетных записей из системы управления доступом следующего вида. Таблица из 2х столбцов, где первый столбец имя учетной записи и второй столбец – роль. При наличии у учетной записи более одной роли, учетная запись записывается в несколько строк:
УЗ1;роль1
УЗ1;роль2
УЗ2;роль1
УЗ3;роль1
..
УЗn;рольn
Также представлена матрица, описывающая запрещенные к совмещения в рамках одной учетной записи роли. Первая строка и первый столбец матрицы/таблицы содержат названия ролей. В случаях если совмещение ролей запрещено в ячейках пересечения столбцов и строк этих ролей ставиться символ «З».
Задача: составить алгоритм анализа таблицы учетных записей для выявления случаев наличия у учетной записи запрещенных к совмещению ролей.

In [ ]:
df = pd.io.excel.read_excel(r'/content/газпром 2023 апрель2.xlsx')
rules = pd.io.excel.read_excel(r'/content/газпром 2023 апрель3.xlsx')

rules = rules.set_index('a')
rules.index.names = [None]
rules

,роль 1,роль 2,роль 3,роль 4
роль 1,NaN,NaN,з,з
роль 2,NaN,NaN,NaN,NaN
роль 3,з,NaN,NaN,NaN
роль 4,з,NaN,NaN,NaN


nan

In [ ]:
df

,УЗ,роль
0,уз1,роль 1
1,уз1,роль 2
2,уз2,роль 3
3,уз3,роль 1
4,уз4,роль 2
5,уз5,роль 3
6,уз6,роль 1
7,уз7,роль 2
8,уз8,роль 3
9,уз9,роль 1


In [ ]:
for i in range(df.shape[0]-1):
  grouped = df.groupby(['УЗ'])


In [ ]:
grouped = df.groupby(['УЗ']).agg(sum).reset_index()
grouped.index.names = [None]
grouped


,УЗ,роль
0,уз1,роль 1роль 2
1,уз10,роль 2
2,уз11,роль 3
3,уз12,роль 1роль 2роль 3роль 4
4,уз13,роль 1
5,уз14,роль 2
6,уз15,роль 3
7,уз16,роль 1
8,уз17,роль 2
9,уз18,роль 3


In [ ]:
grouped = df.groupby(['УЗ']).agg(sum).reset_index()
grouped.index.names = [None]
n=6
users_incompatibility_roles = pd.DataFrame()

def incompatibility_check(roles):
  key = True
  for i in range(len(roles)):
    for j in range(i, len(roles)):
      if rules[roles[i]][roles[j]] == 'з':
        key = False
  return key

for i in range(grouped.shape[0]):
  str = grouped['роль'][i]
  grouped['роль'][i] =  [str[i:i+n] for i in range(0, len(str), n)]

for i in range(grouped.shape[0]):
  if (incompatibility_check(grouped['роль'][i])) == False:
    users_incompatibility_roles = users_incompatibility_roles.append(grouped.loc[i])

users_incompatibility_roles

<ipython-input-206-fa456380e2f5>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  users_incompatibility_roles = users_incompatibility_roles.append(grouped.loc[i])
<ipython-input-206-fa456380e2f5>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  users_incompatibility_roles = users_incompatibility_roles.append(grouped.loc[i])


,УЗ,роль
3,уз12,"[роль 1, роль 2, роль 3, роль 4]"
27,уз34,"[роль 1, роль 2, роль 3]"
